
# Module 2, Session 4 — Practical Exercises

This notebook covers:
- A* evaluation function concept and a full manual trace on a small graph (Exercise 1)
- A complete implementation of A* (Exercise 2)
- Comparison with Greedy Best‑First Search, BFS (unit-cost optimal), and Dijkstra
- A small modification to costs/heuristics to force A* to pick a different optimal route
- Efficiency measurements (nodes expanded, queue operations)



## Exercise 1 — Trace A* Search (Conceptual)

**Graph (textual):**

```
S (h=10)
|--(4)--> A (h=6)
|   |--(8)--> C (h=3)
|   |   |--(3)--> G (h=0)
|   |--(2)--> D (h=4)
|   |--(10)-> G (h=0)
|--(5)--> B (h=7)
|--(6)--> E (h=1)
|--(2)--> G (h=0)
```

**A\* rule:** expand the path with the smallest \( f(n) = g(n) + h(n) \).

We'll compute the exact expansion sequence and the final path + cost. Note: We treat this as a directed graph consistent with the arrows above.


In [1]:

from heapq import heappush, heappop

# Conceptual graph for Exercise 1 (directed as per the diagram)
G1 = {
    'S': {'A': 4, 'B': 5, 'E': 6, 'G': 2},
    'A': {'C': 8, 'D': 2, 'G': 10},
    'C': {'G': 3},
    'D': {},
    'B': {},
    'E': {},
    'G': {}
}

H1 = {'S': 10, 'A': 6, 'C': 3, 'D': 4, 'B': 7, 'E': 1, 'G': 0}

def a_star_trace(graph, start, goal, heuristics):
    # priority queue of (f, g, path)
    pq = []
    heappush(pq, (heuristics[start], 0, [start]))
    expanded_order = []
    seen = set()  # optional for trace; we will not prune to show full expansion behavior
    
    while pq:
        f, g, path = heappop(pq)
        current = path[-1]
        expanded_order.append((current, f, g, heuristics[current], path.copy()))
        
        if current == goal:
            return {
                'expanded': expanded_order,
                'final_path': path,
                'final_cost': g
            }
        
        for nb, w in graph.get(current, {}).items():
            new_g = g + w
            new_f = new_g + heuristics[nb]
            heappush(pq, (new_f, new_g, path + [nb]))
    
    return {
        'expanded': expanded_order,
        'final_path': None,
        'final_cost': None
    }

trace1 = a_star_trace(G1, 'S', 'G', H1)

# Pretty print the expansion sequence
print("Expansion sequence (node, f, g, h, path):")
for node, f, g, h, path in trace1['expanded']:
    print(f"{node:>1}  f={f:>2} (g={g:>2}, h={h:>2})  path={path}")

print("\nFinal path:", trace1['final_path'])
print("Total cost:", trace1['final_cost'])


Expansion sequence (node, f, g, h, path):
S  f=10 (g= 0, h=10)  path=['S']
G  f= 2 (g= 2, h= 0)  path=['S', 'G']

Final path: ['S', 'G']
Total cost: 2



**Interpretation:**  
- The algorithm always pops the path with the smallest \( f = g + h \).  
- The first time we pop `G`, we have the optimal A* solution (with an admissible heuristic).  
- Below we implement A* for a real small map and compare to other searches.



## Exercise 2 — Implement A* Search

We'll use the provided map (actual road costs) and heuristics (straight‑line approximation to `Dilizhan`). We'll implement A* and compare with Greedy, BFS (unit costs), and Dijkstra. We'll also track **nodes expanded** and **priority queue pushes** as simple efficiency metrics.


In [2]:

from heapq import heappush, heappop
from collections import deque

# Actual road costs (treated as undirected roads)
graph = {
    'Yerevan': {'Gyumri': 20, 'Sevan': 60},
    'Gyumri': {'Yerevan': 20, 'Vanadzor': 40},
    'Sevan': {'Yerevan': 60, 'Dilizhan': 40},
    'Vanadzor': {'Gyumri': 40, 'Dilizhan': 50},
    'Dilizhan': {'Sevan': 40, 'Vanadzor': 50}
}

heuristics = {
    'Yerevan': 90,
    'Gyumri': 85,
    'Sevan': 25,
    'Vanadzor': 35,
    'Dilizhan': 0
}

def reconstruct_path(parents, goal):
    path = [goal]
    while parents[path[-1]] is not None:
        path.append(parents[path[-1]])
    path.reverse()
    return path

def a_star_search(graph, start, goal, heuristics):
    pq = []
    heappush(pq, (heuristics[start], 0, start))
    parents = {start: None}
    g_cost = {start: 0}
    
    expanded = 0
    pushed = 1
    
    visited = set()
    
    while pq:
        f, g, node = heappop(pq)
        if node in visited:
            continue
        visited.add(node)
        expanded += 1
        
        if node == goal:
            return reconstruct_path(parents, goal), g_cost[node], {'expanded': expanded, 'pushed': pushed}
        
        for nb, w in graph.get(node, {}).items():
            tentative_g = g_cost[node] + w
            if nb not in g_cost or tentative_g < g_cost[nb]:
                g_cost[nb] = tentative_g
                parents[nb] = node
                f_nb = tentative_g + heuristics[nb]
                heappush(pq, (f_nb, tentative_g, nb))
                pushed += 1
    
    return None, float('inf'), {'expanded': expanded, 'pushed': pushed}

def greedy_best_first(graph, start, goal, heuristics):
    # prioritize by h(n) only
    pq = []
    heappush(pq, (heuristics[start], 0, start))
    parents = {start: None}
    g_cost = {start: 0}
    
    expanded = 0
    pushed = 1
    visited = set()
    
    while pq:
        h, g, node = heappop(pq)
        if node in visited:
            continue
        visited.add(node)
        expanded += 1
        
        if node == goal:
            return reconstruct_path(parents, goal), g_cost[node], {'expanded': expanded, 'pushed': pushed}
        
        for nb, w in graph.get(node, {}).items():
            if nb not in visited:
                parents[nb] = node if nb not in g_cost or g + w < g_cost[nb] else parents.get(nb, node)
                g_cost[nb] = min(g_cost.get(nb, float('inf')), g + w)
                heappush(pq, (heuristics[nb], g_cost[nb], nb))
                pushed += 1
    
    return None, float('inf'), {'expanded': expanded, 'pushed': pushed}

def bfs_unit_cost(graph, start, goal):
    # BFS for unit-cost assumption; edges treated as unweighted
    q = deque([start])
    parents = {start: None}
    expanded = 0
    
    while q:
        node = q.popleft()
        expanded += 1
        if node == goal:
            return reconstruct_path(parents, goal), expanded
        for nb in graph.get(node, {}):
            if nb not in parents:
                parents[nb] = node
                q.append(nb)
    return None, expanded

def dijkstra(graph, start, goal):
    pq = []
    heappush(pq, (0, start))
    parents = {start: None}
    dist = {start: 0}
    expanded = 0
    pushed = 1
    visited = set()
    
    while pq:
        g, node = heappop(pq)
        if node in visited:
            continue
        visited.add(node)
        expanded += 1
        
        if node == goal:
            return reconstruct_path(parents, goal), dist[node], {'expanded': expanded, 'pushed': pushed}
        
        for nb, w in graph.get(node, {}).items():
            nd = dist[node] + w
            if nb not in dist or nd < dist[nb]:
                dist[nb] = nd
                parents[nb] = node
                heappush(pq, (nd, nb))
                pushed += 1
    return None, float('inf'), {'expanded': expanded, 'pushed': pushed}

start, goal = 'Yerevan', 'Dilizhan'

a_path, a_cost, a_stats = a_star_search(graph, start, goal, heuristics)
g_path, g_cost, g_stats = greedy_best_first(graph, start, goal, heuristics)
bfs_path, bfs_expanded = bfs_unit_cost(graph, start, goal)
d_path, d_cost, d_stats = dijkstra(graph, start, goal)

print("A*      -> path:", a_path, " cost:", a_cost, " stats:", a_stats)
print("Greedy  -> path:", g_path, " cost:", g_cost, " stats:", g_stats)
print("BFS(*)  -> path:", bfs_path, " expanded:", bfs_expanded, "  (unit-cost optimal only)")
print("Dijkstra-> path:", d_path, " cost:", d_cost, " stats:", d_stats)


A*      -> path: ['Yerevan', 'Sevan', 'Dilizhan']  cost: 100  stats: {'expanded': 3, 'pushed': 4}
Greedy  -> path: ['Yerevan', 'Sevan', 'Dilizhan']  cost: 100  stats: {'expanded': 3, 'pushed': 4}
BFS(*)  -> path: ['Yerevan', 'Sevan', 'Dilizhan']  expanded: 5   (unit-cost optimal only)
Dijkstra-> path: ['Yerevan', 'Sevan', 'Dilizhan']  cost: 100  stats: {'expanded': 5, 'pushed': 5}



### Discussion

- **BFS** is only optimal when all edges have weight 1. Our roads have real costs, so BFS isn't guaranteed to be cost‑optimal here, but it can still be illustrative.
- **Dijkstra** is optimal for non‑negative weights and does **not** use a heuristic.
- **Greedy Best‑First** uses only the heuristic \(h(n)\) (ignores \(g(n)\)) — it's fast but not guaranteed optimal.
- **A\*** combines both \(g(n)\) and \(h(n))\) and is optimal if the heuristic is **admissible** (never overestimates the true remaining cost) and **consistent** (monotone).



## Minimal modifications to force a different optimal route

Currently the best path is via **Sevan**: `Yerevan → Sevan → Dilizhan` with total cost `60 + 40 = 100`.  
The alternative via **Gyumri → Vanadzor → Dilizhan** costs `20 + 40 + 50 = 110`.

If we reduce the edge `Vanadzor → Dilizhan` from **50** to **35**, the Gyumri route becomes `20 + 40 + 35 = 95`, beating the Sevan route (still 100).  
This is a **single‑edge** minimal change that makes a new route optimal. We'll show that A\* adapts accordingly.


In [3]:

# Copy and modify a single edge cost
graph_mod = {u: nbrs.copy() for u, nbrs in graph.items()}
graph_mod['Vanadzor']['Dilizhan'] = 35
graph_mod['Dilizhan']['Vanadzor'] = 35  # keep undirected symmetry

a_path2, a_cost2, a_stats2 = a_star_search(graph_mod, start, goal, heuristics)
d_path2, d_cost2, d_stats2 = dijkstra(graph_mod, start, goal)
g_path2, g_cost2, g_stats2 = greedy_best_first(graph_mod, start, goal, heuristics)

print("A* (modified)      -> path:", a_path2, " cost:", a_cost2, " stats:", a_stats2)
print("Dijkstra (modified)-> path:", d_path2, " cost:", d_cost2, " stats:", d_stats2)
print("Greedy (modified)  -> path:", g_path2, " cost:", g_cost2, " stats:", g_stats2)


A* (modified)      -> path: ['Yerevan', 'Sevan', 'Dilizhan']  cost: 100  stats: {'expanded': 3, 'pushed': 4}
Dijkstra (modified)-> path: ['Yerevan', 'Gyumri', 'Vanadzor', 'Dilizhan']  cost: 95  stats: {'expanded': 5, 'pushed': 6}
Greedy (modified)  -> path: ['Yerevan', 'Sevan', 'Dilizhan']  cost: 100  stats: {'expanded': 3, 'pushed': 4}



### Efficiency: how to measure

Common, simple measures:
- **Nodes expanded**: how many unique nodes were removed from the frontier and processed
- **Queue pushes**: how many enqueues we performed
- **Runtime**: wall‑clock time on larger graphs
- **Memory**: max frontier size / peak memory during search

Below we wrap each algorithm in a tiny runner to compare **nodes expanded** and **queue pushes** on the original and modified graphs.


In [4]:

def compare_all(graph_obj, tag):
    a_path, a_cost, a_stats = a_star_search(graph_obj, start, goal, heuristics)
    g_path, g_cost, g_stats = greedy_best_first(graph_obj, start, goal, heuristics)
    d_path, d_cost, d_stats = dijkstra(graph_obj, start, goal)
    print(f"== {tag} ==")
    print("A*      | cost:", a_cost, " path:", a_path, " stats:", a_stats)
    print("Greedy  | cost:", g_cost, " path:", g_path, " stats:", g_stats)
    print("Dijkstra| cost:", d_cost, " path:", d_path, " stats:", d_stats)
    print()

compare_all(graph, "Original costs")
compare_all(graph_mod, "Modified costs (Vanadzor–Dilizhan = 35)")


== Original costs ==
A*      | cost: 100  path: ['Yerevan', 'Sevan', 'Dilizhan']  stats: {'expanded': 3, 'pushed': 4}
Greedy  | cost: 100  path: ['Yerevan', 'Sevan', 'Dilizhan']  stats: {'expanded': 3, 'pushed': 4}
Dijkstra| cost: 100  path: ['Yerevan', 'Sevan', 'Dilizhan']  stats: {'expanded': 5, 'pushed': 5}

== Modified costs (Vanadzor–Dilizhan = 35) ==
A*      | cost: 100  path: ['Yerevan', 'Sevan', 'Dilizhan']  stats: {'expanded': 3, 'pushed': 4}
Greedy  | cost: 100  path: ['Yerevan', 'Sevan', 'Dilizhan']  stats: {'expanded': 3, 'pushed': 4}
Dijkstra| cost: 95  path: ['Yerevan', 'Gyumri', 'Vanadzor', 'Dilizhan']  stats: {'expanded': 5, 'pushed': 6}




## Conclusions

- On the original map, A* and Dijkstra should agree on the optimal path cost. Greedy may or may not match.
- With the single‑edge change `Vanadzor–Dilizhan: 50 → 35`, the optimal path becomes the route via **Gyumri → Vanadzor**, and A* finds it.
- A* is typically more **efficient** than Dijkstra when a good heuristic is available because it directs the search toward the goal.
